## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, sqrt, pow
import pyspark.sql.functions as f

## Config

In [2]:
spark = SparkSession.builder.master("local[*]").appName("app").getOrCreate()

weather_dataset_paths = [
    "out_2017.txt",
    "out_2018.txt",
    "out_2019.txt",
    "out_2020.txt"
]

stocks_dataset_path = "MS1.txt"

number_stocks_and_weather = 20

## Functions

In [3]:
def weather_transforms(path):
    # Read data
    df = spark.read \
        .option("header", "true") \
        .option("dateFormat", "yyyy-mm-dd") \
        .csv(path)
    # Split temperature into variables
    df = df.withColumn('TEMP', f.split(df['TMP'], ' ').getItem(0)) 
    # Drop unnecessary columns
    df = df.drop(
        "splitcount", "LOCATION", "WIND", "TMP", "DEW", "SLP", "tmp_quality"
    )

    return df

# Weather data

In [24]:
from pyspark.sql import Window
from pyspark.sql.types import *

import sys

# Load and concatenate all weather data
union = weather_transforms(weather_dataset_paths[0])
for weather_dataset_path in weather_dataset_paths[1:]:
    union = union.union(weather_transforms(weather_dataset_path))

# Select stations which easy start date and then first 1000 of those
weather_df = union.filter(union.DATE == "2017-01-01") \
    .select('STATION_ID') \
    .limit(number_stocks_and_weather)

# Filter these over the complete dataset
filtered_union = weather_df.join(union, 'STATION_ID')

# Fill the easy start stations with min and max date
weather_df = weather_df \
    .withColumn("min_date", f.lit("2017-01-01").cast('date')) \
    .withColumn("max_date", f.lit("2019-12-31").cast('date'))

# Expand to add all days
weather_df = weather_df \
    .withColumn('DATE', f.explode(f.expr('sequence(min_date, max_date, interval 1 day)'))) \
    .drop("min_date", "max_date")

# Left join to make sure all dates are there (missing dates will get null)
weather_df = weather_df \
    .join(filtered_union, ["STATION_ID", "DATE"], "left") \
    .sort(['STATION_ID', 'DATE'])
weather_df = weather_df \
    .withColumn("TEMP" , weather_df.TEMP.cast('float'))
weather_df = weather_df.replace([999, 9999], None)

#weather_df = weather_df.na.fill(20) # TODO

# window_ff = Window.partitionBy('STATION_ID')\
#                .orderBy('DATE')\
#                .rowsBetween(-sys.maxsize, 0)

window_ff = Window.partitionBy('STATION_ID')\
               .orderBy('DATE')\
               .rowsBetween(Window.unboundedPreceding,Window.currentRow)

window_bf = Window.partitionBy('STATION_ID')\
               .orderBy('DATE')\
               .rowsBetween(Window.currentRow,Window.unboundedFollowing)
               
               
# window_bf = Window.partitionBy('STATION_ID')\
#                .orderBy('DATE')\
#                .rowsBetween(0, sys.maxsize)
        
def interpol(x, x_prev, x_next, y_prev, y_next, y):
    # return FloatType(2.0)
    if x_prev == x_next:
        return y
    else:
        m = (y_next-y_prev)/(x_next-x_prev)
        y_interpol = y_prev + m * (x - x_prev)
        return y_interpol

def interpol_middle(y, y_prev, y_next):
    if y == y_prev:
        return y
    else:
        return (y_prev + y_next) / 2.0

#Forward and backwards filling, takes around 6 minutes to complete
read_last = f.last(weather_df['TEMP'], ignorenulls=True).over(window_ff)
readtime_last = f.last(weather_df['DATE'], ignorenulls=True).over(window_ff)

read_next = f.first(weather_df['TEMP'], ignorenulls=True).over(window_bf)
readtime_next = f.first(weather_df['DATE'], ignorenulls=True).over(window_bf)

# add the columns to the dataframe
df_filled = weather_df.withColumn('readvalue_ff', read_last.cast('float'))\
                        .withColumn('readtime_ff', readtime_last)\
                        .withColumn('readvalue_bf', read_next.cast('float'))\
                        .withColumn('readtime_bf', readtime_next)
            
interpol_udf = f.udf(interpol, FloatType())   
interpol_middle_udf = f.udf(interpol_middle, FloatType())
## Set Date To Unix Time
from pyspark.sql.functions import unix_timestamp
df_filled = df_filled.withColumn('unix', unix_timestamp('DATE'))


# df_filled = df_filled.withColumn('TEMP_INTERPOL', interpol_udf('unix', 'readtime_ff', 'readtime_bf', 'readvalue_ff', 'readvalue_bf', 'TEMP'))\
#                     .drop('readtime_ff', 'readtime_bf')\

df_filled = df_filled.withColumn('TEMP_INTERPOL', interpol_middle_udf('TEMP','readvalue_ff', 'readvalue_bf'))\
                    .drop('readtime_ff', 'readtime_bf')\

# UNCOMMENT
df_filled.persist()

# df_filled.schema
df_filled.limit(10).show()
# print(f"Number of rows: {weather_df.count()}")
df_filled.filter(df_filled.TEMP.isNull()).limit(10).show()

+-----------+----------+------+------------+------------+----------+-------------+
| STATION_ID|      DATE|  TEMP|readvalue_ff|readvalue_bf|      unix|TEMP_INTERPOL|
+-----------+----------+------+------------+------------+----------+-------------+
|70259526559|2017-01-01| -50.0|       -50.0|       -50.0|1483225200|        -50.0|
|70259526559|2017-01-02|-130.0|      -130.0|      -130.0|1483311600|       -130.0|
|70259526559|2017-01-03|-130.0|      -130.0|      -130.0|1483398000|       -130.0|
|70259526559|2017-01-04|-150.0|      -150.0|      -150.0|1483484400|       -150.0|
|70259526559|2017-01-05|-130.0|      -130.0|      -130.0|1483570800|       -130.0|
|70259526559|2017-01-06|-120.0|      -120.0|      -120.0|1483657200|       -120.0|
|70259526559|2017-01-07|-170.0|      -170.0|      -170.0|1483743600|       -170.0|
|70259526559|2017-01-08|-160.0|      -160.0|      -160.0|1483830000|       -160.0|
|70259526559|2017-01-09|-140.0|      -140.0|      -140.0|1483916400|       -140.0|
|702

# Stock data

In [58]:
# Load data
stocks_df = spark.read \
    .option("header", "false") \
    .csv(stocks_dataset_path)

# Rename columns and drop volume
stocks_df = stocks_df.selectExpr(
    '_c0 AS STOCK',
    '_c1 AS DATE',
    '_c2 AS PRICE',
    '_c3 AS VOLUME'
)
stocks_df = stocks_df.drop("VOLUME")

# Select 1000 stocks with appropriate years
df_stocks_selection = stocks_df \
    .filter((stocks_df.DATE.contains("2017") | stocks_df.DATE.contains("2018") | stocks_df.DATE.contains("2019")))
df_stocks_selection_filtered = df_stocks_selection \
    .filter(df_stocks_selection.DATE == "01/02/2017") \
    .select('STOCK') \
    .limit(number_stocks_and_weather)
selected_stocks = df_stocks_selection_filtered \
    .join(df_stocks_selection, 'STOCK')

# 
stocks_df = df_stocks_selection_filtered \
    .withColumn("min_date", f.lit("2017-01-01").cast('date')) \
    .withColumn("max_date", f.lit("2019-12-31").cast('date'))
stocks_df = stocks_df \
    .withColumn('DATE', f.explode(f.expr('sequence(min_date, max_date, interval 1 day)'))) \
    .drop("min_date", "max_date")

# 
modifiedDF = selected_stocks \
    .withColumn("DATE", f.to_date("DATE", "MM/dd/yyyy")) \
    .dropDuplicates(["STOCK", "DATE"])
stocks_df = stocks_df \
    .join(modifiedDF, ["Stock", "DATE"], "left") \
    .sort(['STOCK', 'DATE'])

stocks_df = stocks_df.withColumn("PRICE", stocks_df.PRICE.cast("int"))

#Forward and backwards filling, takes around 6 minutes to complete
w_forward = Window.partitionBy().orderBy('Stock').rowsBetween(Window.unboundedPreceding,Window.currentRow)
w_backward = Window.partitionBy().orderBy('Stock').rowsBetween(Window.currentRow,Window.unboundedFollowing)
w_forward2 = Window.partitionBy().orderBy('Stock').rowsBetween(Window.unboundedPreceding,Window.currentRow)
w_backward2 = Window.partitionBy().orderBy('Stock').rowsBetween(Window.currentRow,Window.unboundedFollowing)
stocks_df = stocks_df.withColumn('Price',f.last('Price',ignorenulls=True).over(w_forward))\
                             .withColumn('Volume',f.last('Volume',ignorenulls=True).over(w_forward2))\
                             .withColumn('Price',f.first('Price',ignorenulls=True).over(w_backward))\
                             .withColumn('Volume',f.first('Volume',ignorenulls=True).over(w_backward2))
                             

stocks_df.persist()

stocks_df.show()
print(f"Number of rows: {stocks_df.count()}")

+--------------------+----------+-----+
|               STOCK|      DATE|PRICE|
+--------------------+----------+-----+
|12173.Asien--Aust...|2017-01-01|   20|
|12173.Asien--Aust...|2017-01-02|20201|
|12173.Asien--Aust...|2017-01-03|20499|
|12173.Asien--Aust...|2017-01-04|19780|
|12173.Asien--Aust...|2017-01-05|19550|
|12173.Asien--Aust...|2017-01-06|19999|
|12173.Asien--Aust...|2017-01-07|   20|
|12173.Asien--Aust...|2017-01-08|   20|
|12173.Asien--Aust...|2017-01-09|19600|
|12173.Asien--Aust...|2017-01-10|19825|
|12173.Asien--Aust...|2017-01-11|19215|
|12173.Asien--Aust...|2017-01-12|19501|
|12173.Asien--Aust...|2017-01-13|19397|
|12173.Asien--Aust...|2017-01-14|   20|
|12173.Asien--Aust...|2017-01-15|   20|
|12173.Asien--Aust...|2017-01-16|19329|
|12173.Asien--Aust...|2017-01-17|19674|
|12173.Asien--Aust...|2017-01-18|19900|
|12173.Asien--Aust...|2017-01-19|19850|
|12173.Asien--Aust...|2017-01-20|19808|
+--------------------+----------+-----+
only showing top 20 rows

Number of rows

# Part 2

In [ ]:
"""
SQL query used:

SELECT sum(X*Y') / (sqrt(sum(X))*sqrt(sum(Y')))
FROM (
    SELECT S.DATE, S.STOCK, W.STATION_ID_1, W.STATION_ID_2, S.PRICE as X, W.Y'
    FROM stocks AS S, (
        SELECT W1.DATE, W1.STATION_ID AS STATION_ID_1, W2.STATION_ID AS STATION_ID_2, avg|max|min(W1.TEMP, W2.TEMP) as W.Y'
        FROM weather AS W1, weather AS W2,
        WHERE W1.DATE = W2.DATE
    ) AS W
    WHERE S.DATE = W.DATE
)
GROUPBY STOCK, STATION_ID_1, STATION_ID_2

"""

In [74]:
# agg_function = f.greatest
# agg_function = f.least
agg_function = lambda col1, col2: (col(col1) + col(col2)) / 2

Y_df = weather_df \
    .withColumnRenamed("STATION_ID", "STATION_ID_1") \
    .withColumnRenamed("TEMP", "TEMP_1") \
    .join(
        weather_df \
            .withColumnRenamed("STATION_ID", "STATION_ID_2") \
            .withColumnRenamed("TEMP", "TEMP_2"), 
        "DATE"
    )
Y_df = Y_df.withColumn("Y'", agg_function("TEMP_1", "TEMP_2"))
Y_df = Y_df.drop("TEMP_1", "TEMP_2")
Y_df.persist()
Y_df.show()
print(f"Number of rows: {Y_df.count()}")

+----------+------------+------------+------+
|      DATE|STATION_ID_1|STATION_ID_2|    Y'|
+----------+------------+------------+------+
|2017-01-01| 02265099999| 94804099999|  85.0|
|2017-01-01| 02265099999| 94677099999|  95.5|
|2017-01-01| 02265099999| 94584099999| 156.5|
|2017-01-01| 02265099999| 83650099999| 127.5|
|2017-01-01| 02265099999| 76749399999| 137.5|
|2017-01-01| 02265099999| 71889099999| -22.0|
|2017-01-01| 02265099999| 71453099999| -20.0|
|2017-01-01| 02265099999| 71450099999| -37.0|
|2017-01-01| 02265099999| 71322099999|-171.5|
|2017-01-01| 02265099999| 70333325518|  31.0|
|2017-01-01| 02265099999| 70259526559| -27.5|
|2017-01-01| 02265099999| 68487099999|  90.0|
|2017-01-01| 02265099999| 63708099999| 112.5|
|2017-01-01| 02265099999| 62398099999|  37.5|
|2017-01-01| 02265099999| 54292099999| -76.0|
|2017-01-01| 02265099999| 43278099999| 115.5|
|2017-01-01| 02265099999| 41862199999|  63.0|
|2017-01-01| 02265099999| 28552099999| -17.0|
|2017-01-01| 02265099999| 06022499

In [79]:
combined_df = stocks_df \
    .withColumnRenamed("PRICE", "X") \
    .join(Y_df, "DATE")

combined_df_grouped = combined_df \
    .groupBy("STOCK", "STATION_ID_1", "STATION_ID_2") \
    .agg(
        f.sqrt(f.sum(f.pow("X", 2))).alias("X_norm"),
        f.sqrt(f.sum(f.pow("Y'", 2))).alias("Y_norm"),
        f.sum(col("X") * col("Y'")).alias("XY")
    )

combined_df_grouped = combined_df_grouped \
    .withColumn("COSINE_SIM", col("XY") / (col("Y_norm") * col("X_norm")))
combined_df_grouped = combined_df_grouped.drop("X_norm", "Y_norm", "XY")
combined_df_grouped.persist()
combined_df_grouped.show()

print(f"Number of rows: {combined_df_grouped.count()}")

+--------------------+------------+------------+-------------------+
|               STOCK|STATION_ID_1|STATION_ID_2|         COSINE_SIM|
+--------------------+------------+------------+-------------------+
|42814.Nordamerika...| 02265099999| 94677099999| 0.8642378548973022|
|4438.Asien--Austr...| 02265099999| 76749399999| 0.7725067131938536|
|43297.Nordamerika...| 02265099999| 43278099999| 0.8487977948316912|
|13784.Europa_Deut...| 06022499999| 94677099999| 0.5193424493119109|
|28549.Futures--In...| 28552099999| 94804099999| 0.7120823821302918|
|32843.Nordamerika...| 28552099999| 62398099999| 0.6929047123707867|
|31474.Nordamerika...| 28552099999| 54292099999|0.11055468034407347|
|41574.Nordamerika...| 41862199999| 71450099999|0.45457949948720755|
|34271.Nordamerika...| 43278099999| 94584099999| 0.5726049863036856|
|13784.Europa_Deut...| 43278099999| 71453099999| 0.4919507413036301|
|38565.Nordamerika...| 62398099999| 71450099999| 0.6591657325739723|
|37239.Nordamerika...| 68487099999

In [62]:
# stocks_df.filter(col("PRICE").isNotNull()).count()